# 아마존 리뷰 데이터를 이용한 추천시스템2

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
electronics_data = pd.read_csv("./ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])

In [3]:
electronics_data.head()

,userId,productId,Rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [4]:
new_df=electronics_data.groupby("productId").filter(lambda x:x['Rating'].count() >=50)

In [5]:
dataframe = pd.DataFrame(new_df)

In [6]:
dataframe.to_csv("./rating.csv",index=False)

In [7]:
new_df.drop(['timestamp'],axis = 1, inplace = True)

In [9]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

1) Data 가져오기

In [ ]:
#new_df = new_df.rename(columns={'Unnamed: 0': ''}).set_index('')
new_df.head()

In [10]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(new_df,reader)

2) Trainset, Testset으로 나누기

In [11]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state=10)

In [16]:
from surprise import SVD

3) SVD를 이용 trainset에 파라미터 맞추기

In [17]:
algo = SVD(n_factors=5)
algo.fit(trainset)

4) Test accuracy 측정

In [18]:
test_pred = algo.test(testset)

In [19]:
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Item-based Model : Test Set
RMSE: 1.2588


1.2588281070364704

Model-based collaborative filtering system

In [20]:
new_df1 = new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values ='Rating', index ='userId', columns ='productId',fill_value =0)
ratings_matrix.head()

productId,0972683275,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,1400532736,1400599997,1400698987,...,B00000JFMK,B00000JHWX,B00000JI4F,B00000JII6,B00000JMUG,B00000JPPI,B00000JSGF,B00000JYLO,B00000JYWQ,B00000K135
userId,,,,,,,,,,,,,,,,,,,,,
A01852072Z7B68UHLI5UG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0266076X6KPZ6CCHGVS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0293130VTX2ZXA70JQS,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A030530627MK66BD8V4LN,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0571176384K8RBNKGF8O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
ratings_matrix.shape

(9832, 76)

In [22]:
X = ratings_matrix.T
X.head()

userId,A01852072Z7B68UHLI5UG,A0266076X6KPZ6CCHGVS,A0293130VTX2ZXA70JQS,A030530627MK66BD8V4LN,A0571176384K8RBNKGF8O,A0590501PZ7HOWJKBGQ4,A0641581307AKT5MAOU0Q,A076219533YHEV2LJO988,A0821988FXKFYX53V4QG,A099626739FNCRNHIKBCG,...,AZWOPBY75SGAM,AZX0ZDVAFMN78,AZX5LAN9JEAFF,AZX7I110AF0W2,AZXKUK895VGSM,AZXP46IB63PU8,AZYTSU42BZ7TP,AZZGJ2KMWB7R,AZZMV5VT9W7Y8,AZZST8OYL5P4Q
productId,,,,,,,,,,,,,,,,,,,,,
0972683275,0,0,5,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1400501466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1400501520,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1400501776,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1400532620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
X.shape

(76, 9832)

In [24]:
X.index

Index(['0972683275', '1400501466', '1400501520', '1400501776', '1400532620',
       '1400532655', '140053271X', '1400532736', '1400599997', '1400698987',
       '3744295508', '6301977173', '7214047977', '8862935293', '9573212919',
       '9575871979', '9625993428', '9888002198', '9966694544', '9983891212',
       '9984984354', '9985511476', 'B000001OM4', 'B000001OM5', 'B000001OMI',
       'B000001ON6', 'B00000DM9W', 'B00000IGBF', 'B00000J05A', 'B00000J061',
       'B00000J08Q', 'B00000J0D2', 'B00000J0D5', 'B00000J0D8', 'B00000J1EJ',
       'B00000J1EP', 'B00000J1EQ', 'B00000J1F3', 'B00000J1GA', 'B00000J1QK',
       'B00000J1QR', 'B00000J1SC', 'B00000J1TX', 'B00000J1U8', 'B00000J1UQ',
       'B00000J1V3', 'B00000J1V5', 'B00000J3NF', 'B00000J3Q7', 'B00000J3UJ',
       'B00000J434', 'B00000J4EY', 'B00000J4FS', 'B00000J4GE', 'B00000J6WY',
       'B00000JBAT', 'B00000JBHP', 'B00000JBPB', 'B00000JCT8', 'B00000JCTO',
       'B00000JD34', 'B00000JDF5', 'B00000JDF6', 'B00000JDHV', 'B00000JFE3',

In [25]:
X1 = X

In [26]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(76, 10)

In [27]:
decomposed_matrix

array([[ 1.48104508e+02, -1.54638077e-01,  2.45492421e-02,
         3.65918078e-04, -1.34194353e-01,  1.41763626e-05,
         4.60094834e-02,  3.56239326e-04,  4.07480155e-03,
        -8.11605698e-02],
       [ 6.80453163e-04,  1.28295155e+00,  1.48057118e+00,
         2.08219047e-01,  8.28975976e-03,  8.86177592e-03,
         3.57551548e-02, -4.42706953e-02,  6.14064555e+01,
         1.89006083e+00],
       [ 4.43992745e-05,  1.01148204e-01,  3.11495322e-01,
         6.08396309e-03,  2.93863491e-03,  1.07948330e-02,
         2.76627510e-02,  7.28417321e-02,  1.50373280e+00,
        -1.55221139e-01],
       [ 5.11095161e-04,  7.71818621e-01,  4.91908718e-01,
        -3.88870040e-02, -5.21206520e-03,  3.50569076e-02,
        -6.95082374e-02, -3.95371574e-01,  4.09948099e+00,
        -1.76756109e+00],
       [ 4.57457239e-04,  7.98231476e-01,  1.08892000e+00,
         9.37849602e-03,  1.39237466e-02,  2.37408145e-02,
         4.53151272e-02, -1.79826507e-01, -6.77194356e-01,
        -1.

In [28]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(76, 76)

In [29]:
X.index[75]

'B00000K135'

In [30]:
i = X.index[75]

product_names = list(X.index)
product_ID = product_names.index(i) #index구하기
product_ID

75

In [31]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(76,)

In [32]:
correlation_product_ID

array([-0.07798847, -0.58739586, -0.68875145, -0.86859516, -0.42414457,
       -0.08768653, -0.03431453, -0.37177437, -0.75376376, -0.96928527,
       -0.82091365,  0.87389967,  0.98690845, -0.62234141, -0.71576502,
        0.16109698,  0.12085007, -0.76534078, -0.56332488,  0.82381604,
       -0.70114599,  0.92571646,  0.71561186, -0.37627057,  0.24092394,
       -0.75067448, -0.06967097, -0.59560398, -0.44620187,  0.2418546 ,
        0.77741421,  0.69347525,  0.2010449 ,  0.23784316,  0.55818289,
        0.5792553 , -0.58770113, -0.65109034, -0.09612698, -0.55761762,
        0.16670621,  0.72066788, -0.41302967, -0.76590921, -0.74458688,
        0.10846642, -0.02416779, -0.20376178,  0.4499624 , -0.35012525,
       -0.58079701,  0.1629437 , -0.7895054 ,  0.75644595, -0.26556154,
        0.78503896, -0.27465277,  0.20263528, -0.20937237, -0.46686576,
        0.16059203, -0.07632837, -0.69319743,  0.02299961,  0.08006751,
       -0.91923725, -0.57271177, -0.82794961, -0.06925699,  0.85

In [33]:
Recommend = list(X.index[correlation_product_ID > 0.65]) #피어슨 상관계수가 0.65보다 큰 상품들의 index의 list return

Recommend.remove(i) # 이미 구입한 상품은 뺀다

Recommend[0:24] #상위 24개 추천

['6301977173',
 '7214047977',
 '9983891212',
 '9985511476',
 'B000001OM4',
 'B00000J08Q',
 'B00000J0D2',
 'B00000J1SC',
 'B00000J4GE',
 'B00000JBAT',
 'B00000JII6']

In [34]:
Recommend

['6301977173',
 '7214047977',
 '9983891212',
 '9985511476',
 'B000001OM4',
 'B00000J08Q',
 'B00000J0D2',
 'B00000J1SC',
 'B00000J4GE',
 'B00000JBAT',
 'B00000JII6']